# Dataset Split

In [ ]:
import shutil

DATASET = 'gtav' # 'synthia' or 'gtav'
I_ROOT = '/home/simone/Datasets/Synthia/RGB' if DATASET == 'synthia' else '/home/simone/Datasets/GTAV/images' 
L_ROOT = '/home/simone/Datasets/Synthia/GT/LABELS' if DATASET == 'synthia' else '/home/simone/Datasets/GTAV/labels'
Z = 7 if DATASET == 'synthia' else 5

In [ ]:
with open('train.txt', 'r') as f:
    lines = f.readlines()
train_ids = [line.strip() for line in lines]
print(len(train_ids))

with open('val.txt', 'r') as f:
    lines = f.readlines()
val_ids = [line.strip() for line in lines]
print(len(val_ids))

In [ ]:
for id in train_ids:
    shutil.copy(f"{I_ROOT}/{id.zfill(Z)}.png", f"{I_ROOT}/train/{id.zfill(Z)}.png")
    shutil.copy(f"{L_ROOT}/{id.zfill(Z)}.png", f"{L_ROOT}/train/{id.zfill(Z)}.png")

for id in val_ids:
    shutil.copy(f"{I_ROOT}/{id.zfill(Z)}.png", f"{I_ROOT}/val/{id.zfill(Z)}.png")
    shutil.copy(f"{L_ROOT}/{id.zfill(Z)}.png", f"{L_ROOT}/val/{id.zfill(Z)}.png")

## DomainMix, CopyPaste, and CutMix

In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# l = imageio.imread(f"../../Datasets/Synthia/GT/LABELS/train/0000000.png")
l = imageio.imread(f"../../Datasets/Synthia/GT/LABELS/train/0000000.png", format='PNG-FI')
# l = imageio.v3.imread(f"../../Datasets/Synthia/GT/LABELS/train/0000000.png")


In [ ]:
l.shape

In [ ]:
np.unique(l[...,0])

In [ ]:
plt.imshow(l[...,0])

In [ ]:
from datasets.synthia import Synthia

train_set = Synthia(
    'train', 0,
    joint_transform=None,
    transform=None,
    target_transform=None,
    target_aux_transform=None,
    dump_images=False,
    cv_split=False,
    image_in=False, 
    max_iters=2975)

# Test Data Labels

In [ ]:
import argparse
import torch
import numpy as np
import random
from __future__ import absolute_import
from __future__ import division
from config import cfg, assert_and_infer_cfg
from utils.misc import prep_experiment
import datasets

def parse_args():
    parser = argparse.ArgumentParser(description='Semantic Segmentation')
    parser.add_argument('--lr', type=float, default=0.01)
    parser.add_argument('--arch', type=str, default='network.deepv3.DeepWV3Plus',
                        help='Network architecture. We have DeepSRNX50V3PlusD (backbone: ResNeXt50) \
                        and deepWV3Plus (backbone: WideResNet38).')
    parser.add_argument('--dataset', nargs='*', type=str, default=['cityscapes'],
                        help='a list of datasets; cityscapes, mapillary, camvid, kitti, gtav, mapillary, synthia')
    parser.add_argument('--image_uniform_sampling', action='store_true', default=False,
                        help='uniformly sample images across the multiple source domains')
    parser.add_argument('--val_dataset', nargs='*', type=str, default=['bdd100k'],
                        help='a list consists of cityscapes, mapillary, gtav, bdd100k, synthia')
    parser.add_argument('--covstat_val_dataset', nargs='*', type=str, default=[],
                        help='a list consists of cityscapes, mapillary, gtav, bdd100k, synthia')
    parser.add_argument('--cv', type=int, default=0,
                        help='cross-validation split id to use. Default # of splits set to 3 in config')
    parser.add_argument('--class_uniform_pct', type=float, default=0,
                        help='What fraction of images is uniformly sampled')
    parser.add_argument('--class_uniform_tile', type=int, default=1024,
                        help='tile size for class uniform sampling')
    parser.add_argument('--coarse_boost_classes', type=str, default=None,
                        help='use coarse annotations to boost fine data with specific classes')

    parser.add_argument('--img_wt_loss', action='store_true', default=False,
                        help='per-image class-weighted loss')
    parser.add_argument('--cls_wt_loss', action='store_true', default=False,
                        help='class-weighted loss')
    parser.add_argument('--batch_weighting', action='store_true', default=False,
                        help='Batch weighting for class (use nll class weighting using batch stats')

    parser.add_argument('--jointwtborder', action='store_true', default=False,
                        help='Enable boundary label relaxation')
    parser.add_argument('--strict_bdr_cls', type=str, default='',
                        help='Enable boundary label relaxation for specific classes')
    parser.add_argument('--rlx_off_iter', type=int, default=-1,
                        help='Turn off border relaxation after specific epoch count')
    parser.add_argument('--rescale', type=float, default=1.0,
                        help='Warm Restarts new learning rate ratio compared to original lr')
    parser.add_argument('--repoly', type=float, default=1.5,
                        help='Warm Restart new poly exp')

    parser.add_argument('--fp16', action='store_true', default=False,
                        help='Use Nvidia Apex AMP')
    parser.add_argument('--local_rank', default=0, type=int,
                        help='parameter used by apex library')

    parser.add_argument('--sgd', action='store_true', default=True)
    parser.add_argument('--adam', action='store_true', default=False)
    parser.add_argument('--amsgrad', action='store_true', default=False)

    parser.add_argument('--freeze_trunk', action='store_true', default=False)
    parser.add_argument('--hardnm', default=0, type=int,
                        help='0 means no aug, 1 means hard negative mining iter 1,' +
                        '2 means hard negative mining iter 2')

    parser.add_argument('--trunk', type=str, default='resnet101',
                        help='trunk model, can be: resnet101 (default), resnet50')
    parser.add_argument('--max_epoch', type=int, default=180)
    parser.add_argument('--max_iter', type=int, default=30000)
    parser.add_argument('--max_cu_epoch', type=int, default=100000,
                        help='Class Uniform Max Epochs')
    parser.add_argument('--start_epoch', type=int, default=0)
    parser.add_argument('--crop_nopad', action='store_true', default=False)
    parser.add_argument('--rrotate', type=int,
                        default=0, help='degree of random roate')
    parser.add_argument('--color_aug', type=float,
                        default=0.0, help='level of color augmentation')
    parser.add_argument('--gblur', action='store_true', default=False,
                        help='Use Guassian Blur Augmentation')
    parser.add_argument('--bblur', action='store_true', default=False,
                        help='Use Bilateral Blur Augmentation')
    parser.add_argument('--lr_schedule', type=str, default='poly',
                        help='name of lr schedule: poly')
    parser.add_argument('--poly_exp', type=float, default=0.9,
                        help='polynomial LR exponent')
    parser.add_argument('--bs_mult', type=int, default=2,
                        help='Batch size for training per gpu')
    parser.add_argument('--bs_mult_val', type=int, default=1,
                        help='Batch size for Validation per gpu')
    parser.add_argument('--crop_size', type=int, default=720,
                        help='training crop size')
    parser.add_argument('--pre_size', type=int, default=None,
                        help='resize image shorter edge to this before augmentation')
    parser.add_argument('--scale_min', type=float, default=0.5,
                        help='dynamically scale training images down to this size')
    parser.add_argument('--scale_max', type=float, default=2.0,
                        help='dynamically scale training images up to this size')
    parser.add_argument('--weight_decay', type=float, default=5e-4)
    parser.add_argument('--momentum', type=float, default=0.9)
    parser.add_argument('--snapshot', type=str, default=None)
    parser.add_argument('--restore_optimizer', action='store_true', default=False)

    parser.add_argument('--city_mode', type=str, default='train',
                        help='experiment directory date name')
    parser.add_argument('--date', type=str, default='default',
                        help='experiment directory date name')
    parser.add_argument('--exp', type=str, default='default',
                        help='experiment directory name')
    parser.add_argument('--tb_tag', type=str, default='',
                        help='add tag to tb dir')
    parser.add_argument('--ckpt', type=str, default='logs/ckpt',
                        help='Save Checkpoint Point')
    parser.add_argument('--tb_path', type=str, default='logs/tb',
                        help='Save Tensorboard Path')
    parser.add_argument('--syncbn', action='store_true', default=False,
                        help='Use Synchronized BN')
    parser.add_argument('--dump_augmentation_images', action='store_true', default=False,
                        help='Dump Augmentated Images for sanity check')
    parser.add_argument('--test_mode', action='store_true', default=False,
                        help='Minimum testing to verify nothing failed, ' +
                        'Runs code for 1 epoch of train and val')
    parser.add_argument('-wb', '--wt_bound', type=float, default=1.0,
                        help='Weight Scaling for the losses')
    parser.add_argument('--maxSkip', type=int, default=0,
                        help='Skip x number of  frames of video augmented dataset')
    parser.add_argument('--scf', action='store_true', default=False,
                        help='scale correction factor')
    parser.add_argument('--dist_url', default='tcp://127.0.0.1:', type=str,
                        help='url used to set up distributed training')

    parser.add_argument('--wt_layer', nargs='*', type=int, default=[0,0,0,0,0,0,0],
                        help='0: None, 1: IW/IRW, 2: ISW, 3: IS, 4: IN (IBNNet: 0 0 4 4 4 0 0)')
    parser.add_argument('--wt_reg_weight', type=float, default=0.0)
    parser.add_argument('--relax_denom', type=float, default=2.0)
    parser.add_argument('--clusters', type=int, default=50)
    parser.add_argument('--trials', type=int, default=10)
    parser.add_argument('--dynamic', action='store_true', default=False)

    parser.add_argument('--image_in', action='store_true', default=False,
                        help='Input Image Instance Norm')
    parser.add_argument('--cov_stat_epoch', type=int, default=5,
                        help='cov_stat_epoch')
    parser.add_argument('--visualize_feature', action='store_true', default=False,
                        help='Visualize intermediate feature')
    parser.add_argument('--use_wtloss', action='store_true', default=False,
                        help='Automatic setting from wt_layer')
    parser.add_argument('--use_isw', action='store_true', default=False,
                        help='Automatic setting from wt_layer')

    parser.add_argument('--kd', action='store_true', default=False)
    parser.add_argument('--weights_dir', type=str, default='./bin/')
    parser.add_argument('--return_list', action='store_true', default=False)
    parser.add_argument('--reproduce', action='store_true', default=False)

    return parser

def init():
    parser = parse_args()
    args = parser.parse_args(["--dataset", DATASET, # "gtav", "bdd100k", "cityscapes", "mapillary", "synthia",
                            "--val_dataset", DATASET, # "gtav", "bdd100k", "cityscapes", "mapillary", "synthia",
                            "--arch", "network.deepv3.DeepR50V3PlusD", "--city_mode", "train", "--sgd", "--lr_schedule", "poly", 
                            "--lr", "0.01", "--poly_exp", "0.9", "--max_cu_epoch", "10000", "--class_uniform_pct", "0.0", 
                            "--class_uniform_tile", "512", "--crop_size", "768", "--scale_min", "0.5", "--scale_max", "2.0", 
                            "--rrotate", "0", "--max_iter", "40000", "--bs_mult", "8", "--gblur", "--color_aug", "0.5", 
                            "--date", "0101", "--exp", "test", "--ckpt", "./logs/", "--tb_path", "./logs/", "--wt_reg_weight", "0.0",
                            "--relax_denom", "0.0", "--cov_stat_epoch", "0", "--wt_layer", "0", "0", "0", "0", "0", "0", "0"])

    # Enable CUDNN Benchmarking optimization
    if args.reproduce:
        random_seed = cfg.RANDOM_SEED  #304
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
        torch.backends.cudnn.deterministic = False
        torch.backends.cudnn.benchmark = False
        np.random.seed(random_seed)
        random.seed(random_seed)
    else:
        torch.backends.cudnn.benchmark = True

    args.world_size = 1
    torch.cuda.set_device(args.local_rank)

    for i in range(len(args.wt_layer)):
        if args.wt_layer[i] == 1:
            args.use_wtloss = True
        if args.wt_layer[i] == 2:
            args.use_wtloss = True
            args.use_isw = True
    return args, parser

In [ ]:
DATASET = 'gtav' # 'synthia' or 'gtav'
args, parser = init()
assert_and_infer_cfg(args)
writer = prep_experiment(args, parser)

### File List

In [ ]:
train_list, val_list, name = datasets.setup_loaders(args)
len(train_list[0]), len(val_list[0]), name

In [ ]:
import imageio

gtav_to_cityscapes = {
    7: 0,
    8: 1, 
    11: 2, 
    12: 3, 
    13: 4, 
    17: 5,
    19: 6, 
    20: 7, 
    21: 8, 
    22: 9, 
    23: 10, 
    24: 11, 
    25: 12,
    26: 13, 
    27: 14, 
    28: 15, 
    31: 16, 
    32: 17, 
    33: 18
    }

synthia_to_cityscapes = {
    0: 255,  # void
    1: 10,   # sky
    2: 2,    # building
    3: 0,    # road
    4: 1,    # sidewalk
    5: 4,    # fence
    6: 8,    # vegetation
    7: 5,    # pole
    8: 13,   # car
    9: 7,    # traffic sign
    10: 11,  # pedestrian - person
    11: 18,  # bicycle
    12: 17,  # motorcycle
    13: 255, # parking-slot
    14: 255, # road-work
    15: 6,   # traffic light
    16: 9,   # terrain
    17: 12,  # rider
    18: 14,  # truck
    19: 15,  # bus
    20: 16,  # train
    21: 3,   # wall
    22: 255  # Lanemarking
    }

def get_classes(lst, dct, fnc):
    cls_old, cls_new = set(), set()
    for _, lp in lst:
        l = np.array(fnc(lp))
        l = l[...,0] if len(l.shape) == 3 else l
        cls_old.update(l.flatten())

        lc = np.full(l.shape, 255, dtype=np.uint8)
        for k, v in dct.items():
            lc[l == k] = v
        cls_new.update(lc.flatten())
    return cls_old, cls_new

In [ ]:
dct = gtav_to_cityscapes if args.dataset == ['gtav'] else synthia_to_cityscapes if args.dataset == ['synthia'] else {}
fnc = imageio.v3.imread if args.dataset == ['gtav'] else lambda f: imageio.v2.imread(f,format='PNG-FI') if args.dataset == ['synthia'] else None
print(get_classes(val_list[0], dct, fnc))
# print(get_classes(train_list[0], dct, fnc))

### Data Loader

In [ ]:
train_loader, val_loaders, train_obj, extra_val_loaders, covstat_val_loaders = datasets.setup_loaders(args)
len(train_loader), len(val_loaders[DATASET])

In [ ]:
cls = set()
for i, (img, lbl, _, _) in enumerate(val_loaders[DATASET]):
    cls.update(lbl.unique().numpy())
print(cls)

In [ ]:
import math
import matplotlib.pyplot as plt
import torch

In [ ]:
optimizer = torch.optim.SGD([torch.Tensor([0])], lr=1e-2, weight_decay=5e-4, momentum=0.9, nesterov=False)
lambda1 = lambda iteration: math.pow(1 - iteration / 40000, 0.9)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)

lrs = []
for _ in range(40000):
    optimizer.step()
    lrs.append(scheduler.get_last_lr())
    scheduler.step()
plt.plot(lrs)
plt.semilogy()
plt.show()

In [ ]:
import numpy as np
miou = np.array([55.65, 37.89, 59.71, 51.93, 56.20])
miou.mean(), miou.std()